In [61]:
!pip3 install llama-index
!pip3 install llama-index-core
!pip3 install llama-index-embeddings-openai
!pip3 install llama-parse
!pip3 install neo4j
!pip3 install llama-index-llms-ollama
!pip3 install ollama

In [2]:
!wget 'https://raw.githubusercontent.com/Joshua-Yu/graph-rag/main/openai%2Bllamaparse/InjuredWorkerGuidebookCalifornia.pdf' -O './insurance.pdf'


--2024-05-13 19:23:16--  https://raw.githubusercontent.com/Joshua-Yu/graph-rag/main/openai%2Bllamaparse/InjuredWorkerGuidebookCalifornia.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3779314 (3.6M) [application/octet-stream]
Saving to: ‘./insurance.pdf’

./insurance.pdf     100%[===================>]   3.60M  22.5MB/s    in 0.2s    

2024-05-13 19:23:16 (22.5 MB/s) - ‘./insurance.pdf’ saved [3779314/3779314]



In [3]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-dSMKpwu0fPsVOHfM3jxSsOYuMhS4ramkq9G1wjpQrHLZk9ny"

# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = "OPENAI-API-KEY"

In [41]:
# Just runs .complete to make sure the LLM is listening
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3", base_url="http://192.168.1.102:11434")

response = llm.complete("Who is Laurie Voss? write in 10 words")
print(response)

Laurie Voss: Librarian, author, and advocate for school libraries.


In [43]:
from llama_index.llms.openai import OpenAI
#from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import  VectorStoreIndex
from llama_index.core import Settings

EMBEDDING_MODEL  = "nomic-embed-text"
GENERATION_MODEL = "gpt-4"

#llm = OpenAI(model=GENERATION_MODEL)
llm = Ollama(model='llama3:8b', base_url="http://192.168.1.102:11434", request_timeout=120.0)

Settings.llm = llm


In [44]:
from llama_parse import LlamaParse

pdf_file_name = './insurance.pdf'

documents = LlamaParse(result_type="markdown").load_data(pdf_file_name)


Started parsing the file under job_id 24974054-19df-4dfc-8ecf-bd7e2dda07e6


In [6]:
# Check loaded documents

print(f"Number of documents: {len(documents)}")

for doc in documents:
    print(doc.doc_id)
    print(doc.text[:500] + '...')


Number of documents: 1
1b55563a-9858-4944-9f15-f8bb8c5db72a
# Workers’ Compensation in California

# A Guidebook for Injured Workers Sixth Edition April 2016
---
# Workers’ Compensation in California

A Guidebook for Injured Workers

Sixth Edition

April 2016
---
This guidebook was designed and produced by the Office of the Director, Department of Industrial Relations (DIR), in consultation with the Division of Workers’ Compensation (DWC), State of California. It is based on the third edition of this guidebook, prepared in 2006 by the Institute for Resea...


In [46]:
# Parse the documents using MarkdownElementNodeParser

from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=llm, num_workers=1)

nodes = node_parser.get_nodes_from_documents(documents)

42it [00:00, 95480.09it/s]
100%|██████████| 42/42 [06:45<00:00,  9.66s/it]


In [47]:
# Convert nodes into objects

base_nodes, objects = node_parser.get_nodes_and_objects(nodes)


In [48]:
import json


# Check parsed node objects 

print(f"Number of nodes: {len(base_nodes)}")

Number of nodes: 60


In [49]:
TABLE_REF_SUFFIX = '_table_ref'
TABLE_ID_SUFFIX  = '_table'

# Check parsed objects 

print(f"Number of objects: {len(objects)}")

for node in objects: 
    print(f"id:{node.node_id}")
    print(f"hash:{node.hash}")
    print(f"parent:{node.parent_node}")
    print(f"prev:{node.prev_node}")
    print(f"next:{node.next_node}")

    # Object is a Table
    if node.node_id[-1 * len(TABLE_REF_SUFFIX):] == TABLE_REF_SUFFIX:

        if node.next_node is not None:
            next_node = node.next_node
        
            print(f"next_node metadata:{next_node.metadata}")
            print(f"next_next_node:{next_next_nod_id}")

            obj_metadata = json.loads(str(next_node.json()))

            print(str(obj_metadata))

            print(f"def:{obj_metadata['metadata']['table_df']}")
            print(f"summary:{obj_metadata['metadata']['table_summary']}")


    print(f"next:{node.next_node}")
    print(f"type:{node.get_type()}")
    print(f"class:{node.class_name()}")
    print(f"content:{node.get_content()[:200]}")
    print(f"metadata:{node.metadata}")
    print(f"extra:{node.extra_info}")
    
    node_json = json.loads(node.json())

    print(f"start_idx:{node_json.get('start_char_idx')}")
    print(f"end_idx:{node_json['end_char_idx']}")

    if 'table_summary' in node_json: 
        print(f"summary:{node_json['table_summary']}")

    print("=====================================")   

Number of objects: 42
id:8abccb32-04dc-4eed-9c07-2d6ba7eb71e4
hash:0319dbeff7961a93f93c7c05696e4f30c0ae9752bcfc3115a571e9825183877a
parent:None
prev:node_id='24b9bfb2-4a93-4cb3-ba87-38d824b8cee1' node_type=<ObjectType.TEXT: '1'> metadata={} hash='d452ba7d0bc7d1c574417b457f257d5bcc867d0fe5f7afe9c2902c2324930854'
next:node_id='1099dbac-1c22-4501-8e4d-89ed4044ce1e' node_type=<ObjectType.TEXT: '1'> metadata={'table_df': "{'What is workers’ compensation?': {0: 'What are the benefits?', 1: 'Can my regular doctor treat me if I get hurt on the job?', 2: 'Can all workers predesignate?', 3: 'How to predesignate', 4: 'Are there different rules for predesignating if my employer or the insurer has a contract with a health care organization (HCO)?', 5: 'What should I do if I get hurt at work or develop a work-related medical problem?', 6: 'How can I avoid getting hurt on the job?', 7: 'Workers’ Compensation Benefits—Examples'}, '2': {0: 2, 1: 2, 2: 3, 3: 3, 4: 4, 5: 4, 6: 4, 7: 5}}", 'table_summary'

In [56]:
from neo4j import GraphDatabase

# Local Neo4j instance
# NEO4J_URL = "bolt://localhost:7687"
# Remote Neo4j instance on AuraDB
NEO4J_URL = "bolt://neo4j-1:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "abc123abc123"
NEO4J_DATABASE = ""

def initialiseNeo4jSchema():
    cypher_schema = [
        "CREATE CONSTRAINT sectionKey IF NOT EXISTS FOR (c:Section) REQUIRE (c.key) IS UNIQUE;",
        "CREATE CONSTRAINT chunkKey IF NOT EXISTS FOR (c:Chunk) REQUIRE (c.key) IS UNIQUE;",
        "CREATE CONSTRAINT documentKey IF NOT EXISTS FOR (c:Document) REQUIRE (c.url_hash) IS UNIQUE;",
        "CREATE VECTOR INDEX `chunkVectorIndex` IF NOT EXISTS FOR (e:Embedding) ON (e.value) OPTIONS { indexConfig: {`vector.dimensions`: 1536, `vector.similarity_function`: 'cosine'}};"
    ]

    driver = GraphDatabase.driver(NEO4J_URL, database=NEO4J_DATABASE, auth=(NEO4J_USER, NEO4J_PASSWORD))

    with driver.session() as session:
        for cypher in cypher_schema:
            session.run(cypher)
    driver.close()

In [57]:
initialiseNeo4jSchema()


In [58]:
driver = GraphDatabase.driver(NEO4J_URL, database=NEO4J_DATABASE, auth=(NEO4J_USER, NEO4J_PASSWORD))

# ================================================
# 1) Save documents

print("Start saving documents to Neo4j...")
i = 0
with driver.session() as session:
    for doc in documents:
        cypher = "MERGE (d:Document {url_hash: $doc_id}) ON CREATE SET d.url=$url;"
        session.run(cypher, doc_id=doc.doc_id, url=doc.doc_id)
        i = i + 1
    session.close()

print(f"{i} documents saved.")

# ================================================
# 2) Save nodes

print("Start saving nodes to Neo4j...")

i = 0
with driver.session() as session:
    for node in base_nodes: 

        # >>1 Create Section node
        cypher  = "MERGE (c:Section {key: $node_id})\n"
        cypher += " FOREACH (ignoreMe IN CASE WHEN c.type IS NULL THEN [1] ELSE [] END |\n"
        cypher += "     SET c.hash = $hash, c.text=$content, c.type=$type, c.class=$class_name, c.start_idx=$start_idx, c.end_idx=$end_idx )\n"
        cypher += " WITH c\n"
        cypher += " MATCH (d:Document {url_hash: $doc_id})\n"
        cypher += " MERGE (d)<-[:HAS_DOCUMENT]-(c);"

        node_json = json.loads(node.json())

        session.run(cypher, node_id=node.node_id, hash=node.hash, content=node.get_content(), type='TEXT', class_name=node.class_name()
                          , start_idx=node_json['start_char_idx'], end_idx=node_json['end_char_idx'], doc_id=node.ref_doc_id)

        # >>2 Link node using NEXT relationship

        if node.next_node is not None: # and node.next_node.node_id[-1*len(TABLE_REF_SUFFIX):] != TABLE_REF_SUFFIX:
            cypher  = "MATCH (c:Section {key: $node_id})\n"    # current node should exist
            cypher += "MERGE (p:Section {key: $next_id})\n"    # previous node may not exist
            cypher += "MERGE (p)<-[:NEXT]-(c);"

            session.run(cypher, node_id=node.node_id, next_id=node.next_node.node_id)

        if node.prev_node is not None:  # Because tables are in objects list, so we need to link from the opposite direction
            cypher  = "MATCH (c:Section {key: $node_id})\n"    # current node should exist
            cypher += "MERGE (p:Section {key: $prev_id})\n"    # previous node may not exist
            cypher += "MERGE (p)-[:NEXT]->(c);"

            if node.prev_node.node_id[-1 * len(TABLE_ID_SUFFIX):] == TABLE_ID_SUFFIX:
                prev_id = node.prev_node.node_id + '_ref'
            else:
                prev_id = node.prev_node.node_id

            session.run(cypher, node_id=node.node_id, prev_id=prev_id)

        i = i + 1
    session.close()

print(f"{i} nodes saved.")

# ================================================
# 3) Save objects

print("Start saving objects to Neo4j...")

i = 0
with driver.session() as session:
    for node in objects:               
        node_json = json.loads(node.json())

        # Object is a Table, then the ????_ref_table object is created as a Section, and the table object is Chunk
        if node.node_id[-1 * len(TABLE_REF_SUFFIX):] == TABLE_REF_SUFFIX:
            if node.next_node is not None:  # here is where actual table object is loaded
                next_node = node.next_node

                obj_metadata = json.loads(str(next_node.json()))

                cypher  = "MERGE (s:Section {key: $node_id})\n"
                cypher += "WITH s MERGE (c:Chunk {key: $table_id})\n"
                cypher += " FOREACH (ignoreMe IN CASE WHEN c.type IS NULL THEN [1] ELSE [] END |\n"
                cypher += "     SET c.hash = $hash, c.definition=$content, c.text=$table_summary, c.type=$type, c.start_idx=$start_idx, c.end_idx=$end_idx )\n"
                cypher += " WITH s, c\n"
                cypher += " MERGE (s) <-[:UNDER_SECTION]- (c)\n"
                cypher += " WITH s MATCH (d:Document {url_hash: $doc_id})\n"
                cypher += " MERGE (d)<-[:HAS_DOCUMENT]-(s);"

                session.run(cypher, node_id=node.node_id, hash=next_node.hash, content=obj_metadata['metadata']['table_df'], type='TABLE'
                                  , start_idx=node_json['start_char_idx'], end_idx=node_json['end_char_idx']
                                  , doc_id=node.ref_doc_id, table_summary=obj_metadata['metadata']['table_summary'], table_id=next_node.node_id)
                
            if node.prev_node is not None:
                cypher  = "MATCH (c:Section {key: $node_id})\n"    # current node should exist
                cypher += "MERGE (p:Section {key: $prev_id})\n"    # previous node may not exist
                cypher += "MERGE (p)-[:NEXT]->(c);"

                if node.prev_node.node_id[-1 * len(TABLE_ID_SUFFIX):] == TABLE_ID_SUFFIX:
                    prev_id = node.prev_node.node_id + '_ref'
                else:
                    prev_id = node.prev_node.node_id
                
                session.run(cypher, node_id=node.node_id, prev_id=prev_id)
                
        i = i + 1
    session.close()

# ================================================
# 4) Create Chunks for each Section object of type TEXT
# If there are changes to the content of TEXT section, the Section node needs to be recreated

print("Start creating chunks for each TEXT Section...")

with driver.session() as session:

    cypher  = "MATCH (s:Section) WHERE s.type='TEXT' \n"
    cypher += "WITH s CALL {\n"
    cypher += "WITH s WITH s, split(s.text, '\n') AS para\n"
    cypher += "WITH s, para, range(0, size(para)-1) AS iterator\n"
    cypher += "UNWIND iterator AS i WITH s, trim(para[i]) AS chunk, i WHERE size(chunk) > 0\n"
    cypher += "CREATE (c:Chunk {key: s.key + '_' + i}) SET c.type='TEXT', c.text = chunk, c.seq = i \n"
    cypher += "CREATE (s) <-[:UNDER_SECTION]-(c) } IN TRANSACTIONS OF 500 ROWS ;"
    
    session.run(cypher)
    
    session.close()


print(f"{i} objects saved.")

print("=================DONE====================")

driver.close()

Start saving documents to Neo4j...
1 documents saved.
Start saving nodes to Neo4j...
60 nodes saved.
Start saving objects to Neo4j...
Start creating chunks for each TEXT Section...
42 objects saved.
=================DONE====================


# Generating and Storing Text Embeddings

In [78]:
import ollama
from ollama import Client
client = Client(host='http://192.168.1.102:11434')
client.embeddings(model='llama3', prompt='The sky is blue because of rayleigh scattering')

{'embedding': [-3.616058588027954,
  0.26187846064567566,
  0.28789177536964417,
  -0.3928515613079071,
  1.602487564086914,
  -1.2847929000854492,
  -0.28260573744773865,
  -0.5155466198921204,
  -2.4808707237243652,
  1.3253742456436157,
  1.7212176322937012,
  -0.9524745345115662,
  -0.7613602876663208,
  2.6409990787506104,
  0.2827461361885071,
  0.5220630764961243,
  -1.3409188985824585,
  2.080493450164795,
  -4.873706340789795,
  0.9225631356239319,
  -4.767677307128906,
  0.018306050449609756,
  0.9042076468467712,
  1.4699585437774658,
  -0.6879159212112427,
  0.99538254737854,
  5.167906761169434,
  0.7671281099319458,
  2.490455150604248,
  -1.202128529548645,
  -0.8251153826713562,
  1.0738463401794434,
  -0.9881787300109863,
  1.4650189876556396,
  4.136751651763916,
  -1.4955487251281738,
  -1.879084825515747,
  -2.689141035079956,
  1.9198271036148071,
  2.518340826034546,
  -2.233539342880249,
  -1.106351375579834,
  -0.04469339922070503,
  -2.418288469314575,
  2.3287

In [90]:
from openai import OpenAI
import ollama

EMBEDDING_MODEL = "nomic-embed-text"

def get_embedding(client, text, model):
    response = client.embeddings(
                    prompt=text,
                    model=model,
                )
    return response["embedding"]

def LoadEmbedding(label, property):
    driver = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USER, NEO4J_PASSWORD), database=NEO4J_DATABASE)
    client = Client(host='http://192.168.1.102:11434')
    
    with driver.session() as session:
        # get chunks in document, together with their section titles
        result = session.run(f"MATCH (ch:{label}) RETURN id(ch) AS id, ch.{property} AS text")
        # call OpenAI embedding API to generate embeddings for each proporty of node
        # for each node, update the embedding property
        count = 0
        for record in result:
            id = record["id"]
            text = record["text"]
            
            # For better performance, text can be batched
            embedding = get_embedding(client, text, EMBEDDING_MODEL)
            
            # key property of Embedding node differentiates different embeddings
            cypher = "CREATE (e:Embedding) SET e.key=$key, e.value=$embedding, e.model=$model"
            cypher = cypher + " WITH e MATCH (n) WHERE id(n) = $id CREATE (n) -[:HAS_EMBEDDING]-> (e)"
            session.run(cypher,key=property, embedding=embedding, id=id, model=EMBEDDING_MODEL) 
            count = count + 1

        session.close()
        
        print("Processed " + str(count) + " " + label + " nodes for property @" + property + ".")
        return count


In [91]:


# For smaller amount (<2000) of text data to embed
LoadEmbedding("Chunk", "text")



Processed 1043 Chunk nodes for property @text.


1043